In [1]:
from update_json import update_json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

%matplotlib inline
init_notebook_mode(connected=True)

In [2]:
'''
run one of the two lines below to either read the data file, or update then read the data file

read_json is faster, but it doesn't update the data file

remove the return_df param from update_json to update the file without reading it
'''
#df = pd.read_json('data.json').T

df = update_json('data.json', return_df=True).T

In [3]:
df['avgCommentLenth'] = df['avgCommentLenth'].astype(float)
df['commentCount'] = df['commentCount'].astype(float)

In [4]:
df = df.reset_index()

In [5]:
df.columns = ['Celebrity', 'avgCommentLength', 'avgKarma', 'commentCount', 'Comments', 'Username', 'avgVader']

In [6]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
Celebrity           51 non-null object
avgCommentLength    51 non-null float64
avgKarma            51 non-null object
commentCount        51 non-null float64
Comments            51 non-null object
Username            51 non-null object
avgVader            51 non-null object
dtypes: float64(2), object(5)
memory usage: 2.9+ KB


,Celebrity,avgCommentLength,avgKarma,commentCount,Comments,Username,avgVader
0,Macaulay Culkin,54.433962,"{'compound': 3581.977358490566, 'ups': 3581.97...",265.0,"[{'text': 'My penis. ', 'length': 10, 'utc_tim...",MacaulayCulkinAMA,"{'compound': 0.14748830188679202, 'neg': 0.053..."
1,Patrick Stewart,182.783784,"{'compound': 2770.5945945945946, 'ups': 2770.5...",37.0,"[{'text': 'Frakes with a beard, preferable, be...",sirpatstew,"{'compound': 0.48746756756756704, 'neg': 0.029..."
2,Arnold Schwarzenegger,249.148352,"{'compound': 1621.5054945054944, 'ups': 1621.5...",364.0,[{'text': 'What an incredible story. Thank you...,GovSchwarzenegger,"{'compound': 0.419281868131868, 'neg': 0.03788..."
3,Bill Gates,308.345528,"{'compound': 3915.8943089430895, 'ups': 3915.8...",246.0,[{'text': 'I would be glad to pass along your ...,thisisbillgates,"{'compound': 0.48653821138211306, 'neg': 0.050..."
4,Snoop Dogg,30.460000,"{'compound': 892.929, 'ups': 892.929, 'douns':...",1000.0,[{'text': 'KBone !! When you coming thru tha d...,Here_Comes_The_King,"{'compound': 0.09898459999999999, 'neg': 0.040..."


In [7]:
counts = df[['Celebrity','commentCount', 'avgCommentLength']]

In [8]:
karma = []
vader = []
for k, v in zip(df['avgKarma'], df['avgVader']):
    karma.append(k['compound'])
    vader.append(k['compound'])
counts['Karma'] = karma
counts['Vader'] = vader

/Users/fred/.dotfiles/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/fred/.dotfiles/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [30]:
data = [
    go.Bar(
        x=df['Celebrity'], 
        y=df['commentCount'], 
        name='Total Comment Count',
        textposition = 'auto'),
    go.Bar(
        x=df['Celebrity'], 
        y=df['avgCommentLength'], 
        name='Avg Comment Length',
        textposition = 'auto'),
#     go.Bar(x=df['Celebrity'], y=karma, name='Net Karma'),
#     go.Bar(x=df['Celebrity'], y=vader, name='Avg Compound Vader Score')
]

layout= go.Layout(
    title= "User's Comment Count vs Average Comment Lenth",
    hovermode= 'closest',
    xaxis= dict(
        title= 'Comment Count',
        ticklen= 5,
        #zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Avg Comment Length (Characters)',
        ticklen= 5,
        #zeroline=False,
        gridwidth= 2
    ),
    showlegend= False
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [20]:
data = [
    go.Scatter(x=df['commentCount'],
               y=df['avgCommentLength'],
               mode= 'markers',
               text= df['Celebrity'],
              )
]

layout= go.Layout(
    title= "User's Comment Count vs Average Comment Lenth",
    hovermode= 'closest',
    xaxis= dict(
        title= 'Comment Count',
        ticklen= 5,
        #zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Avg Comment Length (Characters)',
        ticklen= 5,
        #zeroline=False,
        gridwidth= 2
    ),
    showlegend= False
)

# iplot(data, filename='basic-bar')

fig=go.Figure(data=data, layout=layout)
iplot(fig)